In [17]:
import psycopg2
import pandas as pd

In [18]:
# подтягивание аккаунта из excel

profile = pd.read_excel("account.profile.xlsx").astype(str)
profile['profile_tg_id'].astype(str)
profile['profile_role_name'].astype(str)
profile['profile_name'].astype(str)


# загрузка в бд schedule.lesson

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in profile.iterrows():
    try:
        cur.execute("""
            INSERT INTO account.profile (profile_tg_id, profile_role_name, profile_name)
            VALUES (%s, %s, %s)
            """,
            (
                row['profile_tg_id'],
                row['profile_role_name'],
                row['profile_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

profile

,profile_tg_id,profile_role_name,profile_name
0,123456789,Сотрудник,Иванов Иван Иванович
1,987654321,Сотрудник,Петрова Анна Сергеевна
2,234567890,Сотрудник,Сидоров Алексей Викторович
3,345678901,Сотрудник,Кузнецова Мария Павловна
4,456789012,Сотрудник,Смирнов Дмитрий Александрович
5,567890123,Сотрудник,Федорова Ольга Николаевна
6,678901234,Сотрудник,Васильев Сергей Владимирович
7,789012345,Сотрудник,Николаева Екатерина Андреевна
8,890123456,Сотрудник,Орлов Игорь Васильевич
9,901234567,Сотрудник,Григорьев Павел Сергеевич


In [19]:
# подтягиваем информацию о расписании из excel

final_res = pd.DataFrame()
for i in range(4):
    exec(f'lessons = pd.read_excel("1 курс_группы_11ТМ_12ИИ_13ИИ_14БТ.xlsx", sheet_name = {i})')
    x = 0 
    for j in range(len(lessons.columns)//13): 
        exec(f'df_{j} = lessons[lessons.columns[{x}:{x+13}].tolist()]')
        exec(f'df_{j}.columns = lessons.columns[:13].tolist()')
        x+=14
    print(i)
    df = pd.concat([df_0,df_1,df_2,df_3,df_4])
    df = df.dropna(how='all')
    df[df.columns[0]] = df[df.columns[0]].fillna(method='ffill')

    result = pd.DataFrame()

    for c in range(1,len(df.columns),3):
        df_copy = df[[df.columns[0],df.columns[c],df.columns[c+1],df.columns[c+2]]]
        df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
        df_date_sub_copy = df_copy.loc[(df_copy[df.columns[c+2]].isna())][[df.columns[0],df.columns[c],df.columns[c+1]]]
        df_copy= df_copy.dropna()
        exec(f'res_{c} = df_copy.merge(df_date_sub_copy, on =[df.columns[0],df.columns[{c}+1]])')
        exec(f'res_{c}["group"] = df.columns[{c}]')
        exec(f'res_{c}.columns = ["Дата занятия","Тип занятия","Время занятия","Аудитория","Предмет","Группа"]')
        exec(f'result = pd.concat([result, res_{c}]).reset_index(drop=True)')
        exec(f'result.dropna(inplace=True)')
    final_res = pd.concat([final_res,result]).reset_index(drop=True)    
final_res['Дата занятия']=final_res['Дата занятия'].str[:-3] + ".2024"


final_res['Время начала занятия'] = final_res['Время занятия'].str.split('\n', expand=True)[0].str.replace('-', ':')
final_res['Время конца занятия'] = final_res['Время занятия'].str.split('\n', expand=True)[1].str.replace('-', ':')
final_res['ссылка'] = 'null'
final_res['Дата занятия'] = pd.to_datetime(final_res['Дата занятия'], format='%d.%m.%Y')
final_res['Время начала занятия'] = final_res['Дата занятия'].astype(str) + ' ' + final_res['Время начала занятия'].astype(str)
final_res['Время конца занятия'] = final_res['Дата занятия'].astype(str) + ' ' + final_res['Время конца занятия'].astype(str)
final_res['id_учителя']='0'

final_res = final_res[['Группа', 'Тип занятия', 'Аудитория', 'ссылка', 'Время начала занятия', 'Время конца занятия', 'id_учителя', 'Предмет']]

final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Лекции'], 'Лекция')
final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Практические занятия'], 'Практическое занятие')
final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Лабораторные занятия'], 'Лабораторное занятие')

# задаем к каждому предмету id препода

final_res.loc[final_res['Предмет']== 'Технологии AR/VR',                                                    ['id_учителя'] ]= '123456789'
final_res.loc[final_res['Предмет']== '3D анатомия - физиология',                                            ['id_учителя'] ]= '987654321'
final_res.loc[final_res['Предмет']== 'Ознакомительная практика',                                            ['id_учителя'] ]= '234567890'
final_res.loc[final_res['Предмет']== 'Медицинская статистика',                                              ['id_учителя'] ]= '345678901'
final_res.loc[final_res['Предмет']== 'Анализ данных на  Python',                                            ['id_учителя'] ]= '456789012'
final_res.loc[final_res['Предмет']== 'Медицинские аппаратно-программные комплексы',                         ['id_учителя'] ]= '567890123'
final_res.loc[final_res['Предмет']== 'Системный анализ и организация здравоохранения',                      ['id_учителя'] ]= '678901234'
final_res.loc[final_res['Предмет']== 'Методы глубокого обучения',                                           ['id_учителя'] ]= '789012345'
final_res.loc[final_res['Предмет']== 'Системы поддержки принятия решений',                                  ['id_учителя'] ]= '890123456'
final_res.loc[final_res['Предмет']== 'Иностранный язык в профессиональной сфере',                           ['id_учителя'] ]= '901234567'
final_res.loc[final_res['Предмет']== 'Основы молекулярной биологии',                                        ['id_учителя'] ]= '112233445'
final_res.loc[final_res['Предмет']== 'Педагогическая практика',                                             ['id_учителя'] ]= '223344556'
final_res.loc[final_res['Предмет']== 'Базовые аспекты биохимии человека',                                   ['id_учителя'] ]= '334455667'
final_res.loc[final_res['Предмет']== 'Методическое сопровождение применения диагностических тест-систем',   ['id_учителя'] ]= '445566778'
final_res.loc[final_res['Предмет']== 'Биоинформатика',                                                      ['id_учителя'] ]= '556677889'
final_res.loc[final_res['Предмет']== 'Основы медицинской микробиологии и иммунологии',                      ['id_учителя'] ]= '667788990'
final_res.loc[final_res['Предмет']== 'Технологическая акселерация',                                         ['id_учителя'] ]= '778899001'


# загрузка в бд schedule.lesson

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in final_res.iterrows():
    try:
        cur.execute("""
            INSERT INTO schedule.lesson (lesson_type_group_name, lesson_type_education_name, lesson_type_education_room_name,
            lesson_type_education_room_link, lesson_date_time_start, lesson_date_time_end,
            lesson_link, lesson_teacher_id, lesson_subject_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['Группа'],
                row['Тип занятия'],
                row['Аудитория'],
                row['ссылка'],
                row['Время начала занятия'],
                row['Время конца занятия'],
                row['ссылка'],
                row['id_учителя'],
                row['Предмет']
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

final_res

0
1
2


C:\Users\pizdn\AppData\Local\Temp\ipykernel_17632\1728095605.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[df.columns[0]] = df[df.columns[0]].fillna(method='ffill')
C:\Users\pizdn\AppData\Local\Temp\ipykernel_17632\1728095605.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
C:\Users\pizdn\AppData\Local\Temp\ipykernel_17632\1728095605.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
C:\Users\pizdn\AppData\Lo

3


,Группа,Тип занятия,Аудитория,ссылка,Время начала занятия,Время конца занятия,id_учителя,Предмет
0,11-ТМ,Лекция,ЭИОС,null,2024-09-02 14:40,2024-09-02 15:25,123456789,Технологии AR/VR
1,11-ТМ,Лекция,ЭИОС,null,2024-09-02 15:30,2024-09-02 16:15,123456789,Технологии AR/VR
2,11-ТМ,Лекция,ЭИОС,null,2024-09-02 16:20,2024-09-02 17:05,987654321,3D анатомия - физиология
3,11-ТМ,Лекция,ЭИОС,null,2024-09-02 17:10,2024-09-02 17:55,987654321,3D анатомия - физиология
4,11-ТМ,Практическое занятие,"ауд. УК.2.212 (УК Гагарина, д. 18)",null,2024-09-02 18:00,2024-09-02 18:45,987654321,3D анатомия - физиология
...,...,...,...,...,...,...,...,...
2224,14-БТ,Лекция,ЭИОС,null,2024-11-29 17:10,2024-11-29 17:55,556677889,Биоинформатика
2225,14-БТ,Практическое занятие,"ауд. УК.2.213 (УК Гагарина, д. 18)",null,2024-11-29 18:00,2024-11-29 18:45,223344556,Педагогическая практика
2226,14-БТ,Практическое занятие,"ауд. УК.2.213 (УК Гагарина, д. 18)",null,2024-11-29 18:50,2024-11-29 19:35,223344556,Педагогическая практика
2227,14-БТ,Практическое занятие,"ауд. УК.2.213 (УК Гагарина, д. 18)",null,2024-11-29 19:40,2024-11-29 20:25,223344556,Педагогическая практика


In [20]:
final_res['Предмет'].unique()

array(['Технологии AR/VR', '3D анатомия - физиология',
       'Ознакомительная практика', 'Медицинская статистика',
       'Анализ данных на  Python',
       'Медицинские аппаратно-программные комплексы',
       'Системный анализ и организация здравоохранения',
       'Методы глубокого обучения', 'Системы поддержки принятия решений',
       'Иностранный язык в профессиональной сфере',
       'Основы молекулярной биологии', 'Педагогическая практика',
       'Базовые аспекты биохимии человека',
       'Методическое сопровождение применения диагностических тест-систем',
       'Биоинформатика', 'Основы медицинской микробиологии и иммунологии',
       'Технологическая акселерация'], dtype=object)

In [21]:
# получение кафедр

type_department = pd.read_excel("teacher_info.type_department.xlsx").astype(str)
type_department['кафедра'].astype(str)


# загрезка кафедр в бд teacher_info.type_department

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in type_department.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.type_department (type_department_name)
            VALUES (%s)
            """,
            (
                row['кафедра'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

In [22]:

teacher = pd.read_excel("teacher_info.teacher.xlsx")


# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in teacher.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.teacher (teacher_profile_tg_id, teacher_name, teacher_job, teacher_department, teacher_adress, teacher_email)
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (
                row['тг айди'],
                row['фио препода'],
                row['должность'],
                row['кафедра'],
                row['адрес'],
                row['рабочая почта'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

teacher

,тг айди,фио препода,должность,кафедра,адрес,рабочая почта
0,123456789,Иванов Иван Иванович,доцент,Кафедра математики,"ул. Ленина, 1",ivanov@mail.ru
1,987654321,Петрова Анна Сергеевна,старший преподаватель,Кафедра физики,"пр. Мира, 10",petrova@mail.ru
2,234567890,Сидоров Алексей Викторович,профессор,Кафедра информатики,"ул. Пушкина, 5",sidorov@mail.ru
3,345678901,Кузнецова Мария Павловна,доцент,Кафедра химии,"ул. Чехова, 12",kuznetsova@mail.ru
4,456789012,Смирнов Дмитрий Александрович,старший преподаватель,Кафедра биологии,"ул. Горького, 3",smirnov@mail.ru
5,567890123,Федорова Ольга Николаевна,профессор,Кафедра истории,"ул. Советская, 7",fedorova@mail.ru
6,678901234,Васильев Сергей Владимирович,доцент,Кафедра экономики,"ул. Тимирязева, 15",vasilyev@mail.ru
7,789012345,Николаева Екатерина Андреевна,старший преподаватель,Кафедра психологии,"ул. Лермонтова, 20",nikolaeva@mail.ru
8,890123456,Орлов Игорь Васильевич,профессор,Кафедра социологии,"ул. Крылатая, 8",orlov@mail.ru
9,901234567,Григорьев Павел Сергеевич,доцент,Кафедра философии,"ул. Садовая, 4",grigoryev@mail.ru


In [23]:

teacher_img = pd.read_excel("teacher_info.image.xlsx")


# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in teacher_img.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.image (teacher_id, name_img)
            VALUES (%s, %s)
            """,
            (
                row['teacher_id'],
                row['name_img'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

teacher_img

,teacher_id,name_img
0,123456789,img1
1,987654321,img2
2,234567890,img3
3,345678901,img4
4,456789012,img5
5,567890123,img6
6,678901234,img7
7,789012345,img8
8,890123456,img9
9,901234567,img10


In [24]:

adress_contacts_type_place = pd.read_excel("adress_contacts.type_place.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_type_place.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.type_place (type_place_name)
            VALUES (%s)
            """,
            (
                row['type_place_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_type_place

Error inserting row 0: duplicate key value violates unique constraint "type_place_pkey"
DETAIL:  Key (type_place_name)=(Административный корпус) already exists.

Error inserting row 1: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 2: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 3: current transaction is aborted, commands ignored until end of transaction block



,type_place_name
0,Административный корпус
1,Учебный корпус
2,Общежитие
3,Кафедры


In [25]:
adress_contacts_place = pd.read_excel("adress_contacts.place.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_place.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.place (place_name, place_time_start, place_time_end, place_phone, place_email, place_adress, place_adress_real)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['place_name'],
                row['place_time_start'],
                row['place_time_end'],
                row['place_phone'],
                row['place_email'],
                row['place_adress'],
                row['place_adress_real'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_place

Error inserting row 0: duplicate key value violates unique constraint "place_pkey"
DETAIL:  Key (place_name)=(Кафе "Уют") already exists.

Error inserting row 1: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 2: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 3: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 4: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 5: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 6: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 7: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 8: current transaction is aborted, commands ignored until end of transaction block

Error inserting row 9: curren

,place_name,place_time_start,place_time_end,place_phone,place_email,place_adress,place_adress_real
0,"Кафе ""Уют""",08:00:00,22:00:00,84951234567,uyut.cafe@example.com,"ул. Счастливая, 10","ул. Счастливая, 10, Москва"
1,"Ресторан ""Вкусно""",12:00:00,23:00:00,84952345678,vkusno.restaurant@example.com,"пр. Ленина, 5","пр. Ленина, 5, Москва"
2,"Библиотека ""Знание""",09:00:00,20:00:00,84953456789,znanie.library@example.com,"ул. Умная, 15","ул. Умная, 15, Москва"
3,"Спортзал ""Фитнес""",06:00:00,22:00:00,84954567890,fitness.gym@example.com,"ул. Спортивная, 20","ул. Спортивная, 20, Москва"
4,"Магазин ""Все для дома""",10:00:00,21:00:00,84955678901,home.store@example.com,"ул. Комфортная, 30","ул. Комфортная, 30, Москва"
5,"Кинотеатр ""Свет""",11:00:00,01:00:00,84956789012,svet.cinema@example.com,"пр. Кино, 8","пр. Кино, 8, Москва"
6,"Парк ""Зеленый""",07:00:00,22:00:00,84957890123,green.park@example.com,"ул. Природная, 5","ул. Природная, 5, Москва"
7,"Музей ""История""",10:00:00,18:00:00,84958901234,history.museum@example.com,"ул. Культурная, 12","ул. Культурная, 12, Москва"
8,"Аптека ""Здоровье""",08:00:00,21:00:00,84959012345,zdravie.pharmacy@example.com,"ул. Лекарственная, 7","ул. Лекарственная, 7, Москва"
9,"Салон ""Красота""",09:00:00,20:00:00,84950123456,beauty.salon@example.com,"ул. Эстетическая, 3","ул. Эстетическая, 3, Москва"


In [26]:
adress_contacts_place_info = pd.read_excel("adress_contacts.place_info.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_place_info.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.place_info (place_info_type_place_name, place_info_place_name)
            VALUES (%s, %s)
            """,
            (
                row['place_info_type_place_name'],
                row['place_info_place_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_place_info

,place_info_type_place_name,place_info_place_name
0,Административный корпус,"Кафе ""Уют"""
1,Учебный корпус,"Ресторан ""Вкусно"""
2,Кафедры,"Библиотека ""Знание"""
3,Учебный корпус,"Спортзал ""Фитнес"""
4,Учебный корпус,"Магазин ""Все для дома"""
5,Общежитие,"Кинотеатр ""Свет"""
6,Общежитие,"Парк ""Зеленый"""
7,Кафедры,"Музей ""История"""
8,Кафедры,"Аптека ""Здоровье"""
9,Учебный корпус,"Салон ""Красота"""
